The Closing Date, after which the Bank may terminate the right of the Borrower to withdraw from the Loan Account in accordance with the General Conditions, is XXXX.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'flash-surge-313319-fb0596b2676b.json'
from google.cloud import language_v1
client = language_v1.LanguageServiceClient()

In [193]:
df = pd.read_pickle('world_bank.pickle')
df.head()

,year,month,day,id,name,file_content,countries,sections,TITLE,LOAN AGREEMENT,Total Amount,Project Name,ARTICLE II,Amount
0,1991,september,12,558671468103155868,conformed-copy--l3361--kolubara-b-thermal-powe...,CONFORMED COPY\n\n ...,"[Serbia, United States]","{'TITLE': (0, 506), 'LOAN AGREEMENT': (506, 25...",CONFORMED COPY\n\n ...,"LOAN AGREEMENT\n\n AGREEMENT, dated Septe...",300000000,Kolubara B Thermal Power and Lignite Mine Project,ARTICLE II\n\n The...,"$300,000,000"
1,2001,may,31,473681468094784501,conformed-copy--l7054--railway-restructuring-p...,CONFORMED COPY\n\nLOAN NUMBER 7054 POL\n\nLoan...,"[Poland, United States]","{'TITLE': (0, 239), 'LOAN AGREEMENT': (239, 15...",CONFORMED COPY\n\nLOAN NUMBER 7054 POL\n\nLoan...,"LOAN AGREEMENT\n\nAGREEMENT, dated May 31, 200...",110000000,Railway Restructuring Project,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"EUR 110,000,000"
2,1991,november,19,892131468028134392,china--beijing-environmental-project-:-loan-34...,LOAN NUMBER 3415 CHA \n\nLoan Agreement \n\n(B...,"[China, United States]","{'TITLE': (0, 228), 'LOAN AGREEMENT': (228, 21...",LOAN NUMBER 3415 CHA \n\nLoan Agreement \n\n(B...,"LOAN AGREEMENT \n\nAGREEMENT, dated Literal 4b...",60100000,Beijing Environmental Project,ARTICLE II \n\nThe Loan \n\nSection 2.01. The ...,"$45,000,000"
3,2013,may,20,945021468224685176,"official-documents-loan-agreement,-l8236-cn-cl...",MENTS\n\nLOAN NUMBER 8236-CN\n\nLoan Agreement...,"[China, United States]","{'TITLE': (0, 267), 'LOAN AGREEMENT': (267, 48...",MENTS\n\nLOAN NUMBER 8236-CN\n\nLoan Agreement...,"LOAN AGREEMENT\n\nAgreement dated ^46 , 2013, ...",150000000,Liaoning Coastal Economic Zone Urban Infrastru...,ARTICLE II - PROJECT\n\n3.01. The Borrower dec...,
4,1991,may,7,107521468303622178,conformed-copy--l3230--third-highway-sector-pr...,CONFORMED COPY\n\nLOAN NUMBER 3230 YU\n\nLoan ...,"[Bosnia and Herzegovina, United States]","{'TITLE': (0, 364), 'LOAN AGREEMENT': (364, 14...",CONFORMED COPY\n\nLOAN NUMBER 3230 YU\n\nLoan ...,"LOAN AGREEMENT\n\nAGREEMENT, dated May 7, 1991...",55000000,Third Highway Sector Project,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"$55,000,000"


In [227]:
Months = ['January', 'February', 'March', 'April','May','June',
          'July','August','September','October','November','December']

In [256]:
def next_three_lines(x):
    index = 0
    for i in range(3):
        if '\n' in x:
            index += x.find('\n')
            x = x[index+1:]
    return index if index else 150

In [273]:
def get_closing(x,print_ = False):
    if not x:
        return None
    month_ = ''
    day_year = ''
    while 'Closing' in x:
        
        index = x.find('Closing')
        if index == -1:         
            return None
        next_index = next_three_lines(x[index:])
        piece = x[index:index+next_index]
        for month in Months:
            if month in piece:
                if print_:
                    print(piece)
                month_ = month
                index = piece.find(month) + len(month)
                break
        if not month_:
            x = x[index + next_index:]
            continue
        for i in range(index, min(index+20, len(piece))):
            if piece[i].isdigit() or piece[i] in [' ', ',', '\n']:
                day_year += piece[i]
            else:
                break
        
        if month_ and day_year:
            break
        else:
            x = x[index + next_index:]

        
    date = month_ + ' ' + day_year.strip().replace('\n', ' ').replace(',','')
    try:      
        res = pd.Series(date.split(' '), index=['closing_month', 'closing_day', 'closing_year'])
        return res
    except:
        return pd.Series(['','',''], index=['closing_month', 'closing_day', 'closing_year'])
get_closing(doc)

closing_month    March
closing_day         31
closing_year      1998
dtype: object

In [278]:
df[['closing_month', 'closing_day', 'closing_year']] = df['file_content'].apply(get_closing)
df.loc[142, 'closing_year'] = '2021'

In [259]:
df.head()

,year,month,day,id,name,file_content,countries,sections,TITLE,LOAN AGREEMENT,Total Amount,Project Name,ARTICLE II,Amount,closing_month,closing_day,closing_year,loan_length in year
0,1991,september,12,558671468103155868,conformed-copy--l3361--kolubara-b-thermal-powe...,CONFORMED COPY\n\n ...,"[Serbia, United States]","{'TITLE': (0, 506), 'LOAN AGREEMENT': (506, 25...",CONFORMED COPY\n\n ...,"LOAN AGREEMENT\n\n AGREEMENT, dated Septe...",300000000,Kolubara B Thermal Power and Lignite Mine Project,ARTICLE II\n\n The...,"$300,000,000",June,30,1997,6.0
1,2001,may,31,473681468094784501,conformed-copy--l7054--railway-restructuring-p...,CONFORMED COPY\n\nLOAN NUMBER 7054 POL\n\nLoan...,"[Poland, United States]","{'TITLE': (0, 239), 'LOAN AGREEMENT': (239, 15...",CONFORMED COPY\n\nLOAN NUMBER 7054 POL\n\nLoan...,"LOAN AGREEMENT\n\nAGREEMENT, dated May 31, 200...",110000000,Railway Restructuring Project,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"EUR 110,000,000",June,30,2004,3.0
2,1991,november,19,892131468028134392,china--beijing-environmental-project-:-loan-34...,LOAN NUMBER 3415 CHA \n\nLoan Agreement \n\n(B...,"[China, United States]","{'TITLE': (0, 228), 'LOAN AGREEMENT': (228, 21...",LOAN NUMBER 3415 CHA \n\nLoan Agreement \n\n(B...,"LOAN AGREEMENT \n\nAGREEMENT, dated Literal 4b...",60100000,Beijing Environmental Project,ARTICLE II \n\nThe Loan \n\nSection 2.01. The ...,"$45,000,000",June,30,1997,6.0
3,2013,may,20,945021468224685176,"official-documents-loan-agreement,-l8236-cn-cl...",MENTS\n\nLOAN NUMBER 8236-CN\n\nLoan Agreement...,"[China, United States]","{'TITLE': (0, 267), 'LOAN AGREEMENT': (267, 48...",MENTS\n\nLOAN NUMBER 8236-CN\n\nLoan Agreement...,"LOAN AGREEMENT\n\nAgreement dated ^46 , 2013, ...",150000000,Liaoning Coastal Economic Zone Urban Infrastru...,ARTICLE II - PROJECT\n\n3.01. The Borrower dec...,,September,30,2018,5.0
4,1991,may,7,107521468303622178,conformed-copy--l3230--third-highway-sector-pr...,CONFORMED COPY\n\nLOAN NUMBER 3230 YU\n\nLoan ...,"[Bosnia and Herzegovina, United States]","{'TITLE': (0, 364), 'LOAN AGREEMENT': (364, 14...",CONFORMED COPY\n\nLOAN NUMBER 3230 YU\n\nLoan ...,"LOAN AGREEMENT\n\nAGREEMENT, dated May 7, 1991...",55000000,Third Highway Sector Project,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"$55,000,000",December,31,1994,3.0


In [260]:
df[df['closing_year'] == '']

,year,month,day,id,name,file_content,countries,sections,TITLE,LOAN AGREEMENT,Total Amount,Project Name,ARTICLE II,Amount,closing_month,closing_day,closing_year,loan_length in year
84,2014,july,10,686011468094190095,official-documents-loan-agreement-for-loan-833...,OFFICIAL\nDOCUMENTS\n\nLOAN NUMBER 8338-YF\n\n...,"[Serbia, United States]","{'TITLE': (0, 196), 'LOAN AGREEMENT': (196, 40...",OFFICIAL\nDOCUMENTS\n\nLOAN NUMBER 8338-YF\n\n...,"LOAN AGREEMENT\n\nAgreement dated JLA i A0 , 2...",29100000,Second Serbia Health Project,ARTICLE II - LOAN\n\n2.01. The Bank agrees to ...,"EUR 29,100,000",,,,5.0
155,2000,august,11,309881468035483571,conformed-copy--l4559--third-national-highways...,CONFORMED COPY\n\nLOAN NUMBER 4559 IN\n\nLoan ...,"[India, United States]","{'TITLE': (0, 209), 'LOAN AGREEMENT': (209, 12...",CONFORMED COPY\n\nLOAN NUMBER 4559 IN\n\nLoan ...,"LOAN AGREEMENT\n\nAGREEMENT, dated August 11, ...",516000000,Third National Highways Project,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"$516,000,000",,,,NaN
172,2011,june,10,778271468334925407,guarantee-agreement-for-loan-8064-ua-conformed...,CONFORMED COPY \n\nLOAN NUMBER 8064-UA \n\nGua...,"[Ukraine, United States]","{'TITLE': (0, 1348), 'ARTICLE I': (1348, 3904)}",CONFORMED COPY \n\nLOAN NUMBER 8064-UA \n\nGua...,None,200000000,Energy Efficiency Project,None,,,,,NaN
233,2013,june,7,503831468103752390,"official-documents-loan-agreement,-l8264-tr-cl...",OFFICIAL\nWOCUMENTSi\n\nLOAN NUMBER 8264-TR\n\...,"[Turkey, United States]","{'TITLE': (0, 238), 'LOAN AGREEMENT': (238, 88...",OFFICIAL\nWOCUMENTSi\n\nLOAN NUMBER 8264-TR\n\...,LOAN AGREEMENT\n\nAgreement dated 3.%Atle- 3s ...,624100000,Competitiveness and Savings Development Policy...,ARTICLE II - LOAN\n\n2.01. The Bank agrees to ...,"EUR 624,100,000",,,,1.0
244,2013,september,18,280651468274168210,official-documents-loan-assumption-agreement-f...,OFFICIAL\nt0UVIENTS\n\nLOAN NUMBER 7922-MA\n\n...,"[Morocco, United States]","{'TITLE': (0, 2708), 'ARTICLE I': (2708, 3132)...",OFFICIAL\nt0UVIENTS\n\nLOAN NUMBER 7922-MA\n\n...,None,105200000,Regional Potable Water Supply Systems Project,ARTICLE II\n\nAssumption\n\nSection 2.01. The ...,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3070,2007,june,29,619341468319137903,loan-agreement-l7469-tr-conformed.txt,CONFORMED COPY \n\nLOAN NUMBER 7469 TU \n\nLoa...,"[Turkey, United States]","{'TITLE': (0, 254), 'LOAN AGREEMENT': (254, 91...",CONFORMED COPY \n\nLOAN NUMBER 7469 TU \n\nLoa...,"LOAN AGREEMENT \n\nAgreement dated June 29, 20...",17700000000,Competitiveness and Employment Development Pol...,ARTICLE II - LOAN \n\n2.01. The Bank agrees to...,"�367,300,000",,,,1.0
3090,1995,january,24,881801468056926864,mexico--financial-sector-technical-assistance-...,LAU S&P SBN\nME \n\n* OFFICIAL \nDOCUMENTS \n\...,[],"{'TITLE': (0, 97), 'LOAN AGREEMENT': (97, 1903...",LAU S&P SBN\nME \n\n* OFFICIAL \nDOCUMENTS \n\...,LOAN AGREEMENT \n\n(Financial Sector Technical...,37400000,Amendment,ARTICLE II \n\nEffective Date; Termination \n\...,,,,,NaN
3107,1998,october,27,843621468032968152,conformed-copy--l4387--pilot-community-infrast...,CONFORMED COPY\n\nLOAN NUMBER 4387 GA\n\nLoan ...,"[France, Gabon, United States]","{'TITLE': (0, 260), 'LOAN AGREEMENT': (260, 86...",CONFORMED COPY\n\nLOAN NUMBER 4387 GA\n\nLoan ...,"LOAN AGREEMENT\n\nAGREEMENT, dated October 27,...",300000000,Pilot Community Infrastructure Works and Capac...,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"FRF 30,400,000",,,,NaN
3110,1995,july,7,323891468054589051,conformed-copy--l3907--second-madras-water-sup...,CONFORMED COPY\n\nLOAN NUMBER 3907- IN\n (Amen...,[],"{'TITLE': (0, 273), 'LOAN AGREEMENT': (273, 27...",CONFORMED COPY\n\nLOAN NUMBER 3907- IN\n (Amen...,"LOAN AGREEMENT\n\nAGREEMENT, dated June 12, 19...",275800000,Amendment,ARTICLE II\n\nSection 2.01 is deleted and repl...,"$6,000,000 $269,800,000",,,,NaN


In [261]:
doc = df.loc[3193, 'file_content']
index = doc.find('Closing')
print(doc[index:index+200])
print('Closing in document:', 'Closing' in doc)


Closing in document: False


Notes:
- Some files has closing date stated in ARTICLE II. Some in Schedule 2. Others dose not have closing date included.
- 144 files do not work.

## Loan length (in years)
Time length between the agreement date and closing date.

In [279]:
def get_loan_length(x):
    try:
        start = int(x['year'])
        end = int(x['closing_year'].strip())
        return end - start
    except:
        return None
df['loan_length in year'] = df.apply(get_loan_length,axis=1)

In [280]:
df.head()

,year,month,day,id,name,file_content,countries,sections,TITLE,LOAN AGREEMENT,Total Amount,Project Name,ARTICLE II,Amount,closing_month,closing_day,closing_year,loan_length in year
0,1991,september,12,558671468103155868,conformed-copy--l3361--kolubara-b-thermal-powe...,CONFORMED COPY\n\n ...,"[Serbia, United States]","{'TITLE': (0, 506), 'LOAN AGREEMENT': (506, 25...",CONFORMED COPY\n\n ...,"LOAN AGREEMENT\n\n AGREEMENT, dated Septe...",300000000,Kolubara B Thermal Power and Lignite Mine Project,ARTICLE II\n\n The...,"$300,000,000",June,30,1997,6.0
1,2001,may,31,473681468094784501,conformed-copy--l7054--railway-restructuring-p...,CONFORMED COPY\n\nLOAN NUMBER 7054 POL\n\nLoan...,"[Poland, United States]","{'TITLE': (0, 239), 'LOAN AGREEMENT': (239, 15...",CONFORMED COPY\n\nLOAN NUMBER 7054 POL\n\nLoan...,"LOAN AGREEMENT\n\nAGREEMENT, dated May 31, 200...",110000000,Railway Restructuring Project,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"EUR 110,000,000",June,30,2004,3.0
2,1991,november,19,892131468028134392,china--beijing-environmental-project-:-loan-34...,LOAN NUMBER 3415 CHA \n\nLoan Agreement \n\n(B...,"[China, United States]","{'TITLE': (0, 228), 'LOAN AGREEMENT': (228, 21...",LOAN NUMBER 3415 CHA \n\nLoan Agreement \n\n(B...,"LOAN AGREEMENT \n\nAGREEMENT, dated Literal 4b...",60100000,Beijing Environmental Project,ARTICLE II \n\nThe Loan \n\nSection 2.01. The ...,"$45,000,000",June,30,1997,6.0
3,2013,may,20,945021468224685176,"official-documents-loan-agreement,-l8236-cn-cl...",MENTS\n\nLOAN NUMBER 8236-CN\n\nLoan Agreement...,"[China, United States]","{'TITLE': (0, 267), 'LOAN AGREEMENT': (267, 48...",MENTS\n\nLOAN NUMBER 8236-CN\n\nLoan Agreement...,"LOAN AGREEMENT\n\nAgreement dated ^46 , 2013, ...",150000000,Liaoning Coastal Economic Zone Urban Infrastru...,ARTICLE II - PROJECT\n\n3.01. The Borrower dec...,,September,30,2018,5.0
4,1991,may,7,107521468303622178,conformed-copy--l3230--third-highway-sector-pr...,CONFORMED COPY\n\nLOAN NUMBER 3230 YU\n\nLoan ...,"[Bosnia and Herzegovina, United States]","{'TITLE': (0, 364), 'LOAN AGREEMENT': (364, 14...",CONFORMED COPY\n\nLOAN NUMBER 3230 YU\n\nLoan ...,"LOAN AGREEMENT\n\nAGREEMENT, dated May 7, 1991...",55000000,Third Highway Sector Project,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"$55,000,000",December,31,1994,3.0


In [264]:
df.columns

Index(['year', 'month', 'day', 'id', 'name', 'file_content', 'countries',
       'sections', 'TITLE', 'LOAN AGREEMENT', 'Total Amount', 'Project Name',
       'ARTICLE II', 'Amount', 'closing_month', 'closing_day', 'closing_year',
       'loan_length in year'],
      dtype='object')

In [281]:
columns = ['year', 'month', 'day', 'id', 'name', 'closing_month', 'closing_day', 'closing_year',
       'loan_length in year']
df[columns].to_pickle('closing_dates.pickle')

In [282]:
df.groupby(['loan_length in year'])['id'].count().reset_index(name = 'count')

,loan_length in year,count
0,-3.0,2
1,-2.0,1
2,0.0,115
3,1.0,297
4,2.0,185
5,3.0,319
6,4.0,478
7,5.0,846
8,6.0,573
9,7.0,203


In [283]:
df[df['loan_length in year'] < 0]

,year,month,day,id,name,file_content,countries,sections,TITLE,LOAN AGREEMENT,Total Amount,Project Name,ARTICLE II,Amount,closing_month,closing_day,closing_year,loan_length in year
82,1999,july,6,606771468276557081,conformed-copy--l3256--national-agricultural-e...,CONFORMED COPY\n\nLOAN NUMBER 3256 COB\n\nLoan...,"[Congo, United States]","{'TITLE': (0, 271), 'LOAN AGREEMENT': (271, 16...",CONFORMED COPY\n\nLOAN NUMBER 3256 COB\n\nLoan...,"LOAN AGREEMENT\n\nAGREEMENT, dated November 29...",100000000,National Agricultural Extension and\nAdaptive ...,ARTICLE II\n\nThe Loan\n\nSection 2.01. The Ba...,"$15,800,000",June,30,1996,-3.0
1059,2004,june,2,885081468041400216,loan-agreement-l4731-id--conformed.txt,CONFORMED COPY\n\n ...,"[India, United States]","{'TITLE': (0, 602), 'LOAN AGREEMENT': (602, 12...",CONFORMED COPY\n\n ...,"LOAN AGREEMENT\n\n AGREEMENT, dated Januar...",51500000,States’ Road Infrastructure Development Techni...,ARTICLE II\n\n ...,"$51,500,000",December,31,2001,-3.0
1649,2000,september,18,738041468279933951,conformed-copy--l3393--lesotho-highlands-water...,CONFORMED COPY\n\n ...,"[Lesotho, Namibia, United States, South Africa]","{'TITLE': (0, 509), 'LOAN AGREEMENT': (509, 16...",CONFORMED COPY\n\n ...,"LOAN AGREEMENT\n\n AGREEMENT, dated Septe...",1500000000,Lesotho Highlands Water Project (Phase 1A,ARTICLE II\n\n The...,"$110,000,000",March,31,1998,-2.0


In [284]:
for i in [82, 142, 1059, 1649]:
    doc = df.loc[i, 'file_content']
    get_closing(doc, print_ = True)

    print('==========\n')

Closing Date shall be June 30, 1996 or such later date as the 
Bank shall establish. The Bank shall promptly notify the Borrower of such later date.

Section 2.04. The

Closing Date is December 31, 202 l. 

Secti

Closing Date shall be December 31, 2001 or such
later date as the Bank shall establish. The Bank shall promptly notify
the Borrower of such later date.

     Section 2.04

Closing Date shall be March 31, 1998, or
such later date as the Bank shall establish. The Bank shall promptly
notify the Borr



Notes:
- Four files has a negative value of the loan length, meaning that the closing date is before the agreement date.
- After checking the orignal file in index 142, we can manually correct the closing date to be December 31, 2021.

In [285]:
df.groupby(['loan_length in year'])['id'].count().reset_index(name = 'count').sort_values(by='count', ascending = False)

,loan_length in year,count
7,5.0,846
8,6.0,573
6,4.0,478
5,3.0,319
3,1.0,297
9,7.0,203
4,2.0,185
2,0.0,115
10,8.0,64
11,9.0,10
